# ISWC 2019 Cell-Entity Annotation (CEA) Challenge

https://www.aicrowd.com/challenges/iswc-2019-cell-entity-annotation-cea-challenge

## Import required libraries

In [1]:
import pandas as pd 
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns',10)
pd.set_option('display.max_rows',8000)
import numpy as np 
from SPARQLWrapper import SPARQLWrapper, JSON, XML, N3
from rdflib import Graph
from langdetect import detect
import re
import csv
from agdistispy.agdistis import Agdistis
ag = Agdistis()

Create target DataFrame which contain columns [Table_id, Column_id, Row_id, DBpedia_entity,Cell_before,Cell_after,Where]

In [20]:
df_target = pd.read_csv("../CEA_Round1_Targets.csv", header=None)
df_target.columns=["Table_id", "Column_id", "Row_id"]

In [21]:
df_target["DBpedia_entity"] = ""
df_target["Cell_before"] = ""
df_target["Cell_after"] = ""
df_target["Where"] = ""

In [22]:
df_target.head()

,Table_id,Column_id,Row_id,DBpedia_entity,Cell_before,Cell_after,Where
0,50245608_0_871275842592178099,0,154,,,,
1,39107734_2_2329160387535788734,1,32,,,,
2,22864497_0_8632623712684511496,0,227,,,,
3,66009064_0_9148652238372261251,0,15,,,,
4,21362676_0_6854186738074119688,1,75,,,,


# Method to find uri for entity 
### 1. SPARQL (SPARQLWrapper) https://github.com/RDFLib/sparqlwrapper
### 2. Agdistis (Python bindings for AGDISTIS - Multilingual Disambiguation of Named Entities Using Linked Data) https://pypi.org/project/agdistispy/

## Method to make preprocessing for entity 

In [28]:
def remove_special_signs(word):
    result = ""
    for w in word:
        if  w!="," and w!='[' and w!=']' and w!='?' and w!='>':
            result += w 
    return result

In [29]:
def contain_brackets(word):
    return  word.split("(")[0]

In [30]:
def delete_last_space(word):
    if word[-1] == " ":
        return word[:-1]
    else: return word

In [32]:
def do_make_for_cell(cell):
    try:
        cell_after = remove_special_signs(cell)
        cell_after = contain_brackets(cell_after)
        cell_after = delete_last_space(cell_after)
        return cell_after
    except:
        return cell

In [31]:
def make_query(entity, sparql):
    sparql.setQuery("""
       PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
       SELECT ?s WHERE {
      {
        ?s rdfs:label "%s"@en.
      }
    }
    """%(entity))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    uri = ""
    
    for result in results["results"]["bindings"]:
        res = result['s']['value']
        if "dbpedia.org" in res:
            uri += res+" "
    return uri 

In [34]:
def find_agdistic(cell):
    x = ag.disambiguate(f"<entity>{cell}</entity>")
    uri = x[0]['disambiguatedURL']
    return uri

In [35]:
def find_uri(df, column_id, row_id, sparql):
    try:
        cell_before = df.iloc[row_id, column_id] 
        cell_after = do_make_for_cell(cell_before)
        uri = make_query(cell_after, sparql)
        if uri == "":
            uri = make_query(cell_before, sparql)
            if uri == "":
                uri = find_agdistic(cell_before)
                where="Agdistis_before"
                if "notInWiki" in uri:
                    uri = find_agdistic(cell_after)
                    if "notInWiki" in uri:
                        uri = ""
                else:
                    where="Agdistis_after"
            else:
                where = "sparql_before"
        else:
            where = "sparql_after"
                    
        return uri, cell_before, cell_after, where
    except:
        return np.nan, cell_before, np.nan, np.nan

In [36]:
def make_for_row(row):
    global first_table
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    table_id = row["Table_id"]
    sniffer = csv.Sniffer().has_header(f"../CEA_Round1/{table_id}.csv")
    if first_table == table_id:
        if sniffer:
                df = pd.read_csv(f"../CEA_Round1/{table_id}.csv")
            
        else: 
            df = pd.read_csv(f"../CEA_Round1/{table_id}.csv", header=None)
            
    else:
        if sniffer:
                df = pd.read_csv(f"../CEA_Round1/{table_id}.csv")
                first_table = table_id
            
        else:
            df = pd.read_csv(f"../CEA_Round1/{table_id}.csv", header=None)
            first_table = table_id
            
    column_id = row["Column_id"]
    row_id = row["Row_id"]
    if sniffer:
        uri, cell_before, cell_after, where = find_uri(df, column_id, row_id-1, sparql)
    else:
        uri, cell_before, cell_after, where = find_uri(df, column_id, row_id, sparql)
    row["DBpedia_entity"]= uri
    row["Cell_before"]=cell_before
    row["Cell_after"]=cell_after
    row["Where"]=where
    return row

In [37]:
df_target = df_target.sort_values("Table_id").reset_index(drop=True)
first_table = df_target["Table_id"].unique()[0]

In [38]:
df_target = df_target.apply(make_for_row, axis=1)

## Saving dataframe to csv 

In [46]:
df_target.to_csv('df_target_sparql_Agdistis.csv', index=False)

## Load ground truth

In [4]:
gt = pd.read_csv('CEA_Round1_gt.csv', header=None)
gt.columns=['Table_id', 'Column_id', 'Row_id', 'Link_gt']

## Merge to dataframe 

In [5]:
merge = pd.merge(df_target, gt, on=['Table_id', 'Column_id', 'Row_id'])

## Method to check if own dbpedia entity is in ground truth links

In [39]:
def check(row):
    try:
        dbpedia = row['DBpedia_entity'].split(" ")
        check = True
        for r in dbpedia:
            if r in row['Link_gt'].split(" "):
                row['Good_url'] = True
                check = False

        if check:
            row['Good_url'] = False
        return row 
    except:
        row['Good_url'] = False
        return row 

In [40]:
merge = merge.apply(check, axis=1)

## Saving to csv 

In [45]:
merge.to_csv('merge_sparql_Agdistis.csv', index=False)

In [42]:
len(merge[merge['Good_url']])

6349

In [43]:
len(merge)

8418

In [44]:
(6349/8418)*100

75.42171537182229

In [2]:
merge = pd.read_csv("merge_sparql_Agdistis.csv")

In [3]:
merge.head()

,Table_id,Column_id,Row_id,DBpedia_entity,Cell_before,Cell_after,Where,Link_gt,Good_url
0,10579449_0_1681126353774891032,1,8,http://dbpedia.org/resource/Dinamalar,Dinamalar,Dinamalar,sparql_after,http://dbpedia.org/resource/Dinamalar http://dbpedia.org/resource/Dinamalar http://dbpedia.org/resource/Dina_Malar http://dbpedia.org/resource/DINAMALAR,True
1,10579449_0_1681126353774891032,1,2,http://dbpedia.org/resource/Dainik_Bhaskar,Dainik Bhaskar,Dainik Bhaskar,sparql_after,http://dbpedia.org/resource/Dainik_Bhaskar http://dbpedia.org/resource/Daily_Bhaskar http://dbpedia.org/resource/Dainik_Bhaskar,True
2,10579449_0_1681126353774891032,1,11,NaN,bbc Hindi,bbc Hindi,Agdistis_before,http://dbpedia.org/resource/BBC_World_Service http://dbpedia.org/resource/BBC_Worldwide_Service http://dbpedia.org/resource/British_Broadcasting_Corporation_World_Service http://dbpedia.org/resource/Bbc_world_service http://dbpedia.org/resource/BBC_Radio_World_Service http://dbpedia.org/resource/Auckland_Radio_Trust http://dbpedia.org/resource/World_Service http://dbpedia.org/resource/World_service http://dbpedia.org/resource/BBC_Caribbean http://dbpedia.org/resource/BBC_Turkish_Section http://dbpedia.org/resource/BBC_Türkçe http://dbpedia.org/resource/BBC_General_Overseas_Service http://dbpedia.org/resource/BBC_world_service http://dbpedia.org/resource/BBC's_Overseas_Service http://dbpedia.org/resource/BBC_France http://dbpedia.org/resource/BBC_World_Service http://dbpedia.org/resource/BBC_Turkish http://dbpedia.org/resource/BBC_Empire_Service http://dbpedia.org/resource/BBC_Overseas_Service http://dbpedia.org/resource/BBC_Para_África http://dbpedia.org/resource/BBC_News_(radio_series) http://dbpedia.org/resource/BBC's_overseas_service http://dbpedia.org/resource/BBC_Worldservice http://dbpedia.org/resource/B.B.C._World_Service,False
3,10579449_0_1681126353774891032,1,16,http://dbpedia.org/resource/People's_Daily,People's Daily (Renmin Ri Bao),People's Daily,sparql_after,http://dbpedia.org/resource/People's_Daily http://dbpedia.org/resource/The_Peoples_Daily http://dbpedia.org/resource/The_People's_Daily http://dbpedia.org/resource/Peoples_Daily_China http://dbpedia.org/resource/Rénmín_Rìbào http://dbpedia.org/resource/Gosou.cn http://dbpedia.org/resource/Jike.com http://dbpedia.org/resource/People.cn http://dbpedia.org/resource/People.com.cn http://dbpedia.org/resource/Renmin_ribao http://dbpedia.org/resource/China_People's_Daily_Online http://dbpedia.org/resource/Chinese_People's_Daily http://dbpedia.org/resource/People’s_Daily http://dbpedia.org/resource/Renmin_Ribao http://dbpedia.org/resource/People_Daily http://dbpedia.org/resource/Peoples_Daily_Online http://dbpedia.org/resource/Jike.cn http://dbpedia.org/resource/People's_Daily_China http://dbpedia.org/resource/People's_Daily_Online http://dbpedia.org/resource/Jike http://dbpedia.org/resource/People's_Daily http://dbpedia.org/resource/China_People's_Daily http://dbpedia.org/resource/人民日报,True
4,10579449_0_1681126353774891032,1,13,http://dbpedia.org/resource/Softpedia,Softpedia,Softpedia,sparql_after,http://dbpedia.org/resource/Softpedia http://dbpedia.org/resource/Autoevolution http://dbpedia.org/resource/Softpedia http://dbpedia.org/resource/SoftNews_NET http://dbpedia.org/resource/Softpedia.com,True
